In [14]:
# Import scraping modules
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Import data manipulation and visualization packages

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib as mpl
import statistics

In [2]:
#The data we are going to import is the NFL passing data from the 2019 season,
#which can be found here. We open the site and pass it to BeautifulSoup with the following:

#url page

url = 'https://www.pro-football-reference.com/years/2020/passing.htm'

#open URL and pass to Beautiful Soup 

html = urlopen(url)

stats_page = BeautifulSoup(html)


In [3]:
#The two BeautifulSoup functions we will use to scrape the page are findAll() and getText()

#findAll(name)
#Parameters
#name -- HTML tags to use to parse webpage
#Returns array of all matches to name tag

#getText()
#Returns text from HTML


#Fetch table headers
column_head = stats_page.findAll('tr')[0]
column_head = [i.getText() for i in column_head.findAll('th')]

print(column_head)
# Fetch Rows, we skip the first row since these are column headers 

table_row = stats_page.findAll('tr')[1:]

qb_stats = []

for i in range(len(table_row)):
    qb_stats.append([col.getText() for col in table_row[i].findAll('td')])

#Let's print to examine the first row which should be the column headers.
print(qb_stats[0])

['Rk', 'Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds', 'NY/A', 'ANY/A', 'Sk%', '4QC', 'GWD']
['Deshaun Watson*', 'HOU', '25', 'QB', '16', '16', '4-12-0', '382', '544', '70.2', '4823', '33', '6.1', '7', '1.3', '221', '77', '8.9', '9.5', '12.6', '301.4', '112.4', '70.5', '49', '293', '7.64', '8.22', '8.3', '', '']


In [4]:
#We have our column heads and our stats_page. Now we can use pandas library to make a dataframe

nfl_df = pd.DataFrame(qb_stats, columns = column_head[1:])

nfl_df.head()

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,Deshaun Watson*,HOU,25,QB,16,16,4-12-0,382,544,70.2,...,301.4,112.4,70.5,49,293,7.64,8.22,8.3,,
1,Patrick Mahomes*,KAN,25,QB,15,15,14-1-0,390,588,66.3,...,316.0,108.2,82.9,22,147,7.53,8.33,3.6,3,3
2,Tom Brady,TAM,43,QB,16,16,11-5-0,401,610,65.7,...,289.6,102.2,72.5,21,143,7.12,7.53,3.3,3,3
3,Matt Ryan,ATL,35,QB,16,16,4-12-0,407,626,65.0,...,286.3,93.3,67.0,41,257,6.48,6.52,6.1,,
4,Josh Allen*,BUF,24,QB,16,16,13-3-0,396,572,69.2,...,284.0,107.2,81.7,26,159,7.33,7.82,4.3,2,3


In [5]:
nfl_df.isnull().sum()

Player    3
Tm        3
Age       3
Pos       3
G         3
GS        3
QBrec     3
Cmp       3
Att       3
Cmp%      3
Yds       3
TD        3
TD%       3
Int       3
Int%      3
1D        3
Lng       3
Y/A       3
AY/A      3
Y/C       3
Y/G       3
Rate      3
QBR       3
Sk        3
Yds       3
NY/A      3
ANY/A     3
Sk%       3
4QC       3
GWD       3
dtype: int64

In [6]:
#Looks like two Yards('Yds') columns: passing yards and yards sacked

nfl_df.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%',
       'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C',
       'Y/G', 'Rate', 'QBR', 'Sk', 'Yds', 'NY/A', 'ANY/A', 'Sk%', '4QC',
       'GWD'],
      dtype='object')

In [7]:
#Change the latter 'yds' to 'Yds Sacked' to avoid confusion 
nfl_df_new = nfl_df.columns.values 

nfl_df_new[-6] = 'Yds_Sacked'
nfl_df_new[10] = 'Yds_Pass'
nfl_df.columns = nfl_df_new 

#New columns look good; no confusion between the two columns
nfl_df.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%',
       'Yds_Pass', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A',
       'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds_Sacked', 'NY/A', 'ANY/A', 'Sk%',
       '4QC', 'GWD'],
      dtype='object')

In [8]:
# Select categories for visualization and analysis 

# Cmp% - completion rate 
# Yds_Pass - passing yards 
# TD - passing touchdowns 
# Int - interceptions 
# Y/A - yards per attempt
# Rate - passer rating 
# Player - Player name 
# Tm - Team name 

cats= ['Cmp%', 'Yds_Pass', 'Int', 'Y/A', 'Rate']

final_df = nfl_df[['Player', 'Tm'] + cats]

final_df.head()

,Player,Tm,Cmp%,Yds_Pass,Int,Y/A,Rate
0,Deshaun Watson*,HOU,70.2,4823,7,8.9,112.4
1,Patrick Mahomes*,KAN,66.3,4740,6,8.1,108.2
2,Tom Brady,TAM,65.7,4633,12,7.6,102.2
3,Matt Ryan,ATL,65.0,4581,11,7.3,93.3
4,Josh Allen*,BUF,69.2,4544,10,7.9,107.2


In [9]:
#Check the data types of final subset used for our analysis 

final_df.dtypes

Player      object
Tm          object
Cmp%        object
Yds_Pass    object
Int         object
Y/A         object
Rate        object
dtype: object

In [10]:
#We must convert the appropriate data into numerical data since we are unable to manipulate objects 

for i in cats:
    final_df[i] = pd.to_numeric(nfl_df[i])
    
#Check to confirm - values have be converted to numerical 
final_df.dtypes

<ipython-input-10-f31334008c33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[i] = pd.to_numeric(nfl_df[i])


Player       object
Tm           object
Cmp%        float64
Yds_Pass    float64
Int         float64
Y/A         float64
Rate        float64
dtype: object

In [11]:
#Pro football reference had put characters next to players names with the following achievements: "* Selected to Pro Bowl, + First-Team All-Pro"
# Will use str.replace() to remove both * and +

final_df['Player'] = final_df['Player'].str.replace('*', '')

final_df['Player'] = final_df['Player'].str.replace('+', '')

final_df.head(10)

<ipython-input-11-49df50332210>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Player'] = final_df['Player'].str.replace('*', '')
<ipython-input-11-49df50332210>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Player'] = final_df['Player'].str.replace('+', '')


,Player,Tm,Cmp%,Yds_Pass,Int,Y/A,Rate
0,Deshaun Watson,HOU,70.2,4823.0,7.0,8.9,112.4
1,Patrick Mahomes,KAN,66.3,4740.0,6.0,8.1,108.2
2,Tom Brady,TAM,65.7,4633.0,12.0,7.6,102.2
3,Matt Ryan,ATL,65.0,4581.0,11.0,7.3,93.3
4,Josh Allen,BUF,69.2,4544.0,10.0,7.9,107.2
5,Justin Herbert,LAC,66.6,4336.0,10.0,7.3,98.3
6,Aaron Rodgers,GNB,70.7,4299.0,5.0,8.2,121.5
7,Kirk Cousins,MIN,67.6,4265.0,13.0,8.3,105.0
8,Russell Wilson,SEA,68.8,4212.0,13.0,7.5,105.1
9,Philip Rivers,IND,68.0,4169.0,11.0,7.7,97.0


In [20]:
# we will use the mean as the cutoff or filter for our analysis: only QBs with > 1200 yds passed
final_df['Yds_Pass'].mean()

#Filter by >1200 passing yards

df_1200 = final_df[final_df['Yds_Pass'] > 1200]


In [22]:
#Create statistical rank by percentile; create cols with rank

for i in cats:
    df_1200[i + '_Rank'] = df_1200[i].rank(pct=True)
    
#Interception ranks need to switch since most interceptions should negatively affect rank 

df_1200['Int_Rank'] = 1 - df_1200['Int_Rank']

df_1200.head()

<ipython-input-22-a2de618c91a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1200[i + '_Rank'] = df_1200[i].rank(pct=True)
<ipython-input-22-a2de618c91a4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1200['Int_Rank'] = 1 - df_1200['Int_Rank']


,Player,Tm,Cmp%,Yds_Pass,Int,Y/A,Rate,Cmp%_Rank,Yds_Pass_Rank,Int_Rank,Y/A_Rank,Rate_Rank
0,Deshaun Watson,HOU,70.2,4823.0,7.0,8.9,112.4,0.944444,1.000000,0.750000,1.000000,0.972222
1,Patrick Mahomes,KAN,66.3,4740.0,6.0,8.1,108.2,0.555556,0.972222,0.819444,0.888889,0.944444
2,Tom Brady,TAM,65.7,4633.0,12.0,7.6,102.2,0.472222,0.944444,0.166667,0.680556,0.777778
3,Matt Ryan,ATL,65.0,4581.0,11.0,7.3,93.3,0.361111,0.916667,0.263889,0.513889,0.416667
4,Josh Allen,BUF,69.2,4544.0,10.0,7.9,107.2,0.916667,0.888889,0.402778,0.833333,0.916667
